In [1]:
#  Use the appropriate install depending on your local directory

##%pip install --quiet -U pip -r ../requirements/requirements-train.txt ../.

#%pip install --quiet -U pip -r ./requirements/rzequirements-train.txt ./.

In [2]:
import pickle
import jax
from hydra import compose, initialize

#export PATH="~/Documents/InstadeepGroupProject/jumanji_routing"
from jumanji.training.setup_train import setup_agent, setup_env
from jumanji.training.utils import first_from_device

/Users/marta/Documents/MSc Artificial Intelligence/Instadeep_routing/Routing-Board-Generation/venv/lib/python3.8/site-packages/jumanji/training/loggers.py:28: NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.
  from neptune import new as neptune


In [12]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import numpy as np
import chex
import jax.numpy as jnp
import jumanji
from typing import Dict

from jumanji.environments import Connector
from jumanji.environments.routing.connector.types import Agent, Observation, State
from jumanji.types import TimeStep
from jumanji.environments.routing.connector.utils import get_position, get_target
from routing_board_generation.board_generation_methods.jax_implementation.board_generation.seed_extension import SeedExtensionBoard
from routing_board_generation.board_generation_methods.numpy_implementation.utils.utils import get_heads_and_targets
from routing_board_generation.interface.board_generator_interface import BoardGenerator, BoardName
from datetime import datetime
from copy import deepcopy

In [13]:
from routing_board_generation.board_generation_methods.numpy_implementation.utils.post_processor_utils_numpy import \
    training_board_from_solved_board, extend_wires, count_detours
from routing_board_generation.board_generation_methods.jax_implementation.utils.post_processor_utils_jax import \
    training_board_from_solved_board_jax, extend_wires_jax

In [14]:
import statistics
from routing_board_generation.benchmarking.benchmarks.empty_board_evaluation import EvaluateEmptyBoard

## Load configs

In [17]:
#with initialize(version_base=None, config_path="../jumanji/training/configs"):
    #cfg = compose(config_name="config.yaml", overrides=["env=connector", "agent=a2c"])
with initialize(version_base=None, config_path="../agent_training/configs"):
    cfg = compose(config_name="config.yaml", overrides=["env=connector", "agent=a2c"])
cfg

{'agent': 'a2c', 'seed': 0, 'logger': {'type': 'terminal', 'save_checkpoint': False, 'name': '${agent}_${env.name}'}, 'env': {'name': 'connector', 'registered_version': 'Connector-v0', 'ic_board': {'generation_type': 'offline_seed_extension', 'board_name': 'none', 'grid_size': 10, 'num_agents': 5}, 'seed_extension': {'randomness': 0, 'two_sided': True, 'extension_iterations': 1, 'extension_steps': 1e+23, 'number_of_boards': 100000}, 'network': {'transformer_num_blocks': 4, 'transformer_num_heads': 8, 'transformer_key_size': 16, 'transformer_mlp_units': [512], 'conv_n_channels': 32}, 'training': {'num_epochs': 500, 'num_learner_steps_per_epoch': 100, 'n_steps': 20, 'total_batch_size': 128}, 'evaluation': {'eval_total_batch_size': 5000, 'greedy_eval_total_batch_size': 5000}, 'a2c': {'normalize_advantage': False, 'discount_factor': 0.99, 'bootstrapping_factor': 0.95, 'l_pg': 1.0, 'l_td': 1.0, 'l_en': 0.01, 'learning_rate': 0.0002}}}

## Load a saved checkpoint

In [18]:
file = "../trained_agents/uniform/training_state"
with open(file,"rb") as f:
    training_state = pickle.load(f)

params = first_from_device(training_state.params_state.params)
#print(params)
env = setup_env(cfg).unwrapped
#print(env)
agent = setup_agent(cfg, env)
#print(agent)
policy = jax.jit(agent.make_policy(params.actor, stochastic = False))
#print(params.num_agents)
#print(policy)


# Roll out a few episodes of their default "uniform" board generation

In [19]:
env_fn = jax.jit(env.reset) # Speed up reset
step_fn = jax.jit(env.step)  # Speed up env.step
GRID_SIZE = 10
NUM_AGENTS = 5
NUM_EPISODES = 10 # 10000

print(datetime.now())
states = []
key = jax.random.PRNGKey(cfg.seed)

connections = []
for episode in range(NUM_EPISODES):  
    key, reset_key = jax.random.split(key)
    #state, timestep = jax.jit(env.reset)(reset_key)
    state, timestep = env_fn(reset_key)
        
        
    while not timestep.last():
        key, action_key = jax.random.split(key)
        observation = jax.tree_util.tree_map(lambda x: x[None], timestep.observation)
        # Two implementations for calling the policy, about equivalent speed
        #action, _ = policy(observation, action_key)
        action, _ = jax.jit(policy)(observation, action_key)
        # Three implementations for updating the state/timestep.  The third is much faster.
        #state, timestep = jax.jit(env.step)(state, action.squeeze(axis=0)) # original jit = 0.32, 52sec/10
        #state, timestep = env.step(state, action.squeeze(axis=0)) # no jit = 0.13, 26sec/10
        state, timestep = step_fn(state, action.squeeze(axis=0)) # jit function = 0.003 5 sec/10, 49sec/100d
        states.append(state)

        
        
    # Freeze the terminal frame to pause the GIF.
    for _ in range(10):
        states.append(state)
        
    #################################################
    # Evaluate the number of wires connected
    num_connected = timestep.extras["num_connections"]
    connections.append(num_connected)

#print(connections)
connections = np.array (connections)
print(f"{NUM_EPISODES} Evaluations: Average connections for default Uniform board = {connections.mean()}, std={connections.std()}")
print(datetime.now())

2023-05-02 10:59:25.711194
10 Evaluations: Average connections for default Uniform board = 3.8, std=0.8717797887081347
2023-05-02 10:59:29.197112


# Roll out a few episodes, extending the wires of each board generators

In [20]:
#WE PROBABLY DON'T NEED TO EXTEND THE WIRES OF EACH ANYMORE BUT KEEP THIS CELL FOR HISTORICAL REASONS

step_fn = jax.jit(env.step) #  Speed up env.step
BOARD_GENERATORS = [
        BoardName.BFS_BASE,
        BoardName.BFS_MIN_BENDS,
        BoardName.BFS_FIFO,
        BoardName.BFS_SHORTEST,
        BoardName.BFS_LONGEST,
        BoardName.RANDOM_WALK,
        BoardName.LSYSTEMS,
        BoardName.WFC,
        BoardName.NUMBERLINK]

GRID_SIZE = 10
NUM_AGENTS = 5
NUM_EPISODES = 1 # 10000

print(datetime.now())
states = []
key = jax.random.PRNGKey(cfg.seed)

for BOARD_GENERATOR in BOARD_GENERATORS:
    print(BOARD_GENERATOR)
    connections = []
    detours_raw = []
    detours_extended = []
    for episode in range(NUM_EPISODES):
   
        key, reset_key = jax.random.split(key)


        ### Copied from ic_rl_training  ##########################
        # Create empty grid.
        grid = jnp.zeros((GRID_SIZE, GRID_SIZE), dtype=jnp.int32)

        board_class = BoardGenerator.get_board_generator(
            board_enum=BOARD_GENERATOR)    
        board = board_class(GRID_SIZE, GRID_SIZE, NUM_AGENTS)
        pins = board.return_training_board()

        ### WIRE EXTENSION #####
        #print("PINS")
        #print(pins)
        board_solved = board.return_solved_board()
        detours_raw.append(count_detours(board_solved, False))
        #print("BOARD SOLVED")
        #print(board_solved)
        board_extended = extend_wires(board_solved)
        detours_extended.append(count_detours(board_extended, False))
        #print("BOARD EXTENDED")
        #print(board_extended)
        pins = training_board_from_solved_board_jax(board_extended)
        #print("PINS EXTENDED")
        #print(pins)
        ### END OF WIRE EXTENSION ####

        starts_flat, targets_flat = get_heads_and_targets(pins)
        starts = jnp.divmod(np.array(starts_flat), GRID_SIZE)
        targets = jnp.divmod(np.array(targets_flat), GRID_SIZE)

        agent_position_values = jax.vmap(get_position)(jnp.arange(NUM_AGENTS))
        agent_target_values = jax.vmap(get_target)(jnp.arange(NUM_AGENTS))

        # Place the agent values at starts and targets.
        grid = grid.at[starts].set(agent_position_values)
        grid = grid.at[targets].set(agent_target_values)

        # Create the agent pytree that corresponds to the grid.
        agents = jax.vmap(Agent)(
            id=jnp.arange(NUM_AGENTS),
            start=jnp.stack(starts, axis=1),
            target=jnp.stack(targets, axis=1),
            position=jnp.stack(starts, axis=1),
        )

        step_count = jnp.array(0, jnp.int32)

        state = State(key=key, grid=grid, step_count=step_count, agents=agents)
        # END OF CODE FROM IC_RL_TRAINING
        ########################################################################################

        
        ###############AMENDMENT 2 FROM CLEMENT TO INITIALIZE TIMESTEP############################
        action_mask = jax.vmap(env._get_action_mask, (0, None))(
            state.agents, state.grid
            )
        observation = Observation(
            grid=env._obs_from_grid(state.grid),
            action_mask=action_mask,
            step_count=state.step_count,
            )
        extras = env._get_extras(state)
        timestep = jumanji.types.restart(
            observation=observation, extras=extras, shape=(env.num_agents,)
            )
        ####################################################################
        
        
        while not timestep.last():
            key, action_key = jax.random.split(key)
            observation = jax.tree_util.tree_map(lambda x: x[None], timestep.observation)
            # Two implementations for calling the policy, about equivalent speed
            #action, _ = policy(observation, action_key)
            action, _ = jax.jit(policy)(observation, action_key)
            # Three implementations for updating the state/timestep.  The third is much faster.
            #state, timestep = jax.jit(env.step)(state, action.squeeze(axis=0)) # original jit = 0.32, 52sec/10
            #state, timestep = env.step(state, action.squeeze(axis=0)) # no jit = 0.13, 26sec/10
            state, timestep = step_fn(state, action.squeeze(axis=0)) # jit function = 0.003 5 sec/10, 49sec/100d
            states.append(state)
        # Freeze the terminal frame to pause the GIF.
        for _ in range(10):
            states.append(state)

        # Evaluate the number of wires connected
        num_connected = timestep.extras["num_connections"]
        connections.append(num_connected)

    connections = np.array(connections)
    print(f"{NUM_EPISODES} Evaluations: Average connections for {BOARD_GENERATOR} = {connections.mean()}, std={connections.std()}")
    detours_raw = np.array(detours_raw)    
    print(f"Average Detours (RAW) = {detours_raw.mean()}, (std = {detours_raw.std()}) ")
    detours_extended = np.array(detours_extended)
    print(f"Average Detours (extended) = {detours_extended.mean()}, (std = {detours_extended.std()}) ")
    print(datetime.now())

2023-05-02 10:59:29.236935
BoardName.BFS_BASE
1 Evaluations: Average connections for bfs_base = 4.0, std=0.0
Average Detours (RAW) = 1.0, (std = 0.0) 
Average Detours (extended) = 3.0, (std = 0.0) 
2023-05-02 10:59:31.373208
BoardName.BFS_MIN_BENDS
1 Evaluations: Average connections for bfs_min_bend = 4.0, std=0.0
Average Detours (RAW) = 11.0, (std = 0.0) 
Average Detours (extended) = 13.0, (std = 0.0) 
2023-05-02 10:59:31.506163
BoardName.BFS_FIFO
1 Evaluations: Average connections for bfs_fifo = 3.0, std=0.0
Average Detours (RAW) = 0.0, (std = 0.0) 
Average Detours (extended) = 7.0, (std = 0.0) 
2023-05-02 10:59:31.670605
BoardName.BFS_SHORTEST
1 Evaluations: Average connections for bfs_short = 5.0, std=0.0
Average Detours (RAW) = 1.0, (std = 0.0) 
Average Detours (extended) = 3.0, (std = 0.0) 
2023-05-02 10:59:31.845211
BoardName.BFS_LONGEST
1 Evaluations: Average connections for bfs_long = 3.0, std=0.0
Average Detours (RAW) = 1.0, (std = 0.0) 
Average Detours (extended) = 6.0, (std

# TEST ALL THE AGENTS ON ALL THE BOARDS

In [ ]:
# TEST ALL THE AGENTS ON ALL THE BOARDS

  
key = jax.random.PRNGKey(cfg.seed)
step_fn = jax.jit(env.step) # Speed up env.step
BOARD_GENERATORS = [
        #BoardName.BFS_BASE,
        ##BoardName.BFS_MIN_BENDS, # Don't use this. It's redundant
        ##BoardName.BFS_FIFO,      # Don't use this. It's redundant
        #BoardName.BFS_SHORTEST,
        #BoardName.BFS_LONGEST,
        #BoardName.RANDOM_WALK,
        #BoardName.RANDOM_SEED,
        BoardName.JAX_SEED_EXTENSION,
        BoardName.LSYSTEMS,
        ##BoardName.WFC,           # Don't use this.  Wave Form Collapse will not be pursued.
        #BoardName.NUMBERLINK
        ]

#agents = ["uniform", "bfs_base", "random_walk", "numberlink", "lsystems"]  #file = "trained_agents/"+agent+"/training_state"
#agents = ["iterations1","iterations2","iterations5","iterations6","iterations8"] #file = "trained_agents/fullsteps_random1/"+agentname+"/training_state"
#agents = ["online_randomseed_random1_ext1"]
#agents = ["online_randomseed_100epochs_random085_ext21", "uniform"]
#agents = ["parallel_randomwalk"]
agents = ["bfs_base"]

GRID_SIZE = 10
NUM_AGENTS = 5
NUM_EPISODES = 5000 # 10000


for agentname in agents:
    file = "../trained_agents/"+agentname+"/training_state"
    #file = "trained_agents/fullsteps_random1/"+agentname+"/training_state"
    print("\nAGENT = ",agentname)
    with open(file,"rb") as f:
        training_state = pickle.load(f)
    params = first_from_device(training_state.params_state.params)
    env = setup_env(cfg).unwrapped
    agent = setup_agent(cfg, env)
    policy = jax.jit(agent.make_policy(params.actor, stochastic = False))
    
    connections_all = []
    detours_all = []
    keys_all = []
    episodes_all = []
    densities_all = []
    diversities_all = []
    print(datetime.now())
    for BOARD_GENERATOR in BOARD_GENERATORS: # use this to test multiple generators
    #BOARD_GENERATOR = BoardName.RANDOM_SEED # use this to test varying randomseed parameters
    #for iterations in range(11, 22): # use this to test varying randomseed parameters
        #print("ITERATIONS = ", iterations) # use this to test varying randomseed parameters
        
        states = []
        connections = []
        print(BOARD_GENERATOR)         
        if (BOARD_GENERATOR == BoardName.JAX_SEED_EXTENSION):  #BoardName.RANDOM_SEED):
            board = SeedExtensionBoard(GRID_SIZE, GRID_SIZE, NUM_AGENTS)
            solved_board_compiled = jax.jit(board.return_solved_board, 
                    static_argnames=['extension_iterations', 'randomness','two_sided','extension_steps'])
            #training_board_compiled = jax.jit(board.return_training_board)
            #starts_targets_compiled = jax.jit(board.generate_starts_ends)

            #extended_board_init = board.return_extended_board  #jax.jit(board.return_extended_board)

        for episode in range(NUM_EPISODES):

            key, reset_key = jax.random.split(key)
            #### Copied from ic_rl_training ###################################
            # Create empty grid.
            grid = jnp.zeros((GRID_SIZE, GRID_SIZE), dtype=jnp.int32)

            board_class = BoardGenerator.get_board_generator(
                board_enum=BOARD_GENERATOR)    

            if (BOARD_GENERATOR == BoardName.JAX_SEED_EXTENSION): #RANDOM_SEED):
                key, subkey = jax.random.split(key)

                solved_board = solved_board_compiled(subkey, randomness = 1.0, two_sided = True, 
                                                     extension_iterations=21, extension_steps=1e23) 
                """  For varying randomseed parameters
                solved_board = solved_board_compiled(subkey, randomness = 0.85, two_sided = True, 
                                     extension_iterations=iterations, extension_steps=1e23) 
                """
                
                #print("SOLVED BOARD")
                #print(solved_board)
                #print(solved_board)
                #print(np.array(solved_board))
                np_solved_board = np.array(solved_board)
                #print("shape=",np_solved_board.shape)
                training_board = training_board_from_solved_board(np.array(solved_board))
                pins = training_board
                #print("TRAINING BOARD")
                #print(training_board)
            
            #elif BOARD_GENERATOR == BoardName.LSYSTEMS:
            #    board = board_class(GRID_SIZE, GRID_SIZE, NUM_AGENTS)
            #    key, subkey = jax.random.split(key)
            #    pins = board.return_training_board(subkey)
            #    solved_board = board.return_solved_board(subkey)

            else:
                board = board_class(GRID_SIZE, GRID_SIZE, NUM_AGENTS)
                #print("SOLVED BOARD")
                #print(board.return_solved_board())
                solved_board = board.return_solved_board()
                pins = board.return_training_board() 


            starts_flat, targets_flat = get_heads_and_targets(pins)
            starts = jnp.divmod(np.array(starts_flat), GRID_SIZE)
            targets = jnp.divmod(np.array(targets_flat), GRID_SIZE)

            agent_position_values = jax.vmap(get_position)(jnp.arange(NUM_AGENTS))
            agent_target_values = jax.vmap(get_target)(jnp.arange(NUM_AGENTS))

            # Place the agent values at starts and targets.
            grid = grid.at[starts].set(agent_position_values)
            grid = grid.at[targets].set(agent_target_values)

            # Create the agent pytree that corresponds to the grid.
            agents = jax.vmap(Agent)(
                id=jnp.arange(NUM_AGENTS),
                start=jnp.stack(starts, axis=1),
                target=jnp.stack(targets, axis=1),
                position=jnp.stack(starts, axis=1),
            )
            step_count = jnp.array(0, jnp.int32)
            state = State(key=key, grid=grid, step_count=step_count, agents=agents)
            ### END OF CODE FROM IC_RL_TRAINING  ##################################

            ### AMENDMENT 2 FROM CLEMENT TO INITIALIZE TIMESTEP #################
            action_mask = jax.vmap(env._get_action_mask, (0, None))(
                state.agents, state.grid
                )
            observation = Observation(
                grid=env._obs_from_grid(state.grid),
                action_mask=action_mask,
                step_count=state.step_count,
                )
            extras = env._get_extras(state)
            timestep = jumanji.types.restart(
                observation=observation, extras=extras, shape=(env.num_agents,)
                )
            ####################################################################
            # Let the agent try to solve each board
            while not timestep.last():
                key, action_key = jax.random.split(key)
                observation = jax.tree_util.tree_map(lambda x: x[None], timestep.observation)
                # Two implementations for calling the policy, about equivalent speed
                #action, _ = policy(observation, action_key)
                action, _ = jax.jit(policy)(observation, action_key)
                # Three implementations for updating the state/timestep.  The third is much faster.
                #state, timestep = jax.jit(env.step)(state, action.squeeze(axis=0)) # original jit = 0.32, 52sec/10
                #state, timestep = env.step(state, action.squeeze(axis=0)) # no jit = 0.13, 26sec/10
                state, timestep = step_fn(state, action.squeeze(axis=0)) # jit function = 0.003 5 sec/10, 49sec/100d
                states.append(state)
            # Freeze the terminal frame to pause the GIF.
            for _ in range(10):
                states.append(state)

            # Evaluate the number of wires connected
            num_connected = timestep.extras["num_connections"]
            connections.append(num_connected)
            connections_all.append(num_connected)
            keys_all.append(key[0])
            detours_all.append(count_detours(np.array(solved_board)))
            episodes_all.append(episode)
            densities_all.append(np.count_nonzero(np.array(solved_board)>0))
            board_evaluator = EvaluateEmptyBoard(np.array(solved_board))
            board_stats = board_evaluator.board_statistics
            diversity = board_stats["heatmap_score_diversity"] 
            diversities_all.append(diversity)
            # End of section for each individual board
            
        connections = np.array(connections)
        print(f"{NUM_EPISODES} Evaluations: Average connections for {BOARD_GENERATOR} = {connections.mean()}, std={connections.std()}")  
        print(datetime.now())
        # End of section for each board type
        
    connections_all = np.array(connections_all)
    print(f"{NUM_EPISODES} Evaluations: Total average connections for {agentname} = {connections_all.mean()}, std={connections_all.std()}")  
    print("CORRELATIONS")
    detours_all = np.array(detours_all)
    keys_all = np.array(keys_all)
    episodes_all = np.array(episodes_all)
    #print("mean keys = ", keys_all.mean())
    #print("mean episodes = ", episodes_all.mean())
    print("Correlation of connections with detours is ", statistics.correlation(connections_all, detours_all))
    print("Correlation of connections with keys is ", statistics.correlation(connections_all, keys_all))
    print("Correlation of connections with episodes is ", statistics.correlation(connections_all, episodes_all))
    print("Correlation of connections with densities is ", statistics.correlation(connections_all, densities_all))
    print("Correlation of connections with diversities is ", statistics.correlation(connections_all, diversities_all))



AGENT =  bfs_base
2023-05-02 10:59:44.772540
BoardName.JAX_SEED_EXTENSION


/Users/marta/Documents/MSc Artificial Intelligence/Instadeep_routing/Routing-Board-Generation/venv/lib/python3.8/site-packages/jax/_src/ops/scatter.py:89: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=int32. In future JAX releases this will result in an error.
  warnings.warn("scatter inputs have incompatible types: cannot safely cast "


EVALUATING EVERYONE ON L-SYSTEMS

AGENT =  uniform
2023-05-02 00:19:21.753036
BoardName.LSYSTEMS
5000 Evaluations: Average connections for lsystems_standard = 4.98, std=0.14422205101855956
2023-05-02 00:25:21.383975
5000 Evaluations: Total average connections for uniform = 4.98, std=0.14422205101855956
CORRELATIONS
Correlation of connections with detours is  -0.01935806893685983
Correlation of connections with keys is  0.005346677659771794
Correlation of connections with episodes is  -0.02031642004850264
Correlation of connections with densities is  -0.009123005946855816
Correlation of connections with diversities is  -0.028970764358208122

AGENT =  bfs_base
2023-05-02 00:25:21.807770
BoardName.LSYSTEMS
5000 Evaluations: Average connections for lsystems_standard = 4.971, std=0.1837362239733907
2023-05-02 00:31:21.863111
5000 Evaluations: Total average connections for bfs_base = 4.971, std=0.1837362239733907
CORRELATIONS
Correlation of connections with detours is  -0.029094437054042795
Correlation of connections with keys is  0.004213078077117279
Correlation of connections with episodes is  0.008364618147003282
Correlation of connections with densities is  -0.018583133729298918
Correlation of connections with diversities is  -0.028214478944084118

AGENT =  random_walk
2023-05-02 00:31:22.287055
BoardName.LSYSTEMS
5000 Evaluations: Average connections for lsystems_standard = 4.9632, std=0.20553773376195428
2023-05-02 00:37:16.381361
5000 Evaluations: Total average connections for random_walk = 4.9632, std=0.20553773376195428
CORRELATIONS
Correlation of connections with detours is  -0.03418112837273609
Correlation of connections with keys is  -0.0017943014669556208
Correlation of connections with episodes is  -0.020961467600573856
Correlation of connections with densities is  -0.06580300202526061
Correlation of connections with diversities is  -0.03744817817784907

AGENT =  numberlink
2023-05-02 00:37:16.809231
BoardName.LSYSTEMS
5000 Evaluations: Average connections for lsystems_standard = 4.9334, std=0.2663915163814343
2023-05-02 00:43:35.281696
5000 Evaluations: Total average connections for numberlink = 4.9334, std=0.2663915163814343
CORRELATIONS
Correlation of connections with detours is  -0.018604440193449458
Correlation of connections with keys is  -0.018489353858393426
Correlation of connections with episodes is  -0.008100066280397018
Correlation of connections with densities is  -0.023229531459681183
Correlation of connections with diversities is  -0.04347433663018336

AGENT =  lsystems
2023-05-02 00:43:35.727887
BoardName.LSYSTEMS
5000 Evaluations: Average connections for lsystems_standard = 4.8818, std=0.3690918042980635
2023-05-02 00:50:09.011681
5000 Evaluations: Total average connections for lsystems = 4.8818, std=0.3690918042980635
CORRELATIONS
Correlation of connections with detours is  -0.019051585907028455
Correlation of connections with keys is  0.00023288046099994157
Correlation of connections with episodes is  -0.011855168147067333
Correlation of connections with densities is  -0.0912969748602712
Correlation of connections with diversities is  -0.05610474760955783

In [ ]:
EVALUATING BFS_BASE AGENT ON ALL BOARDS (EXCEPT RANDOMSEED)

AGENT =  bfs_base
2023-05-01 22:57:50.027729
BoardName.BFS_BASE
5000 Evaluations: Average connections for bfs_base = 4.421, std=0.7663935020601362
2023-05-01 23:05:04.410579
BoardName.BFS_SHORTEST
5000 Evaluations: Average connections for bfs_short = 4.2684, std=0.8245977443578172
2023-05-01 23:13:30.757064
BoardName.BFS_LONGEST
5000 Evaluations: Average connections for bfs_long = 4.6306, std=0.6123264162193234
2023-05-01 23:21:41.659476
BoardName.RANDOM_WALK
5000 Evaluations: Average connections for random_walk = 4.5446, std=0.6794194875038542
2023-05-01 23:29:54.634114
BoardName.LSYSTEMS
5000 Evaluations: Average connections for lsystems_standard = 4.9746, std=0.17763682050746124
2023-05-01 23:36:47.631969
BoardName.NUMBERLINK
5000 Evaluations: Average connections for numberlink = 3.7492, std=1.0434075713737179
2023-05-01 23:52:26.303030



In [ ]:
EVALUATING AN AGENT TRAINED ON JAX PARALLEL RANDOMWALK

AGENT =  parallel_randomwalk
2023-05-01 19:33:49.935760
BoardName.BFS_BASE
5000 Evaluations: Average connections for bfs_base = 4.383, std=0.7672750484669758
2023-05-01 19:41:10.288775
BoardName.BFS_SHORTEST
5000 Evaluations: Average connections for bfs_short = 4.2244, std=0.8539582191184765
2023-05-01 19:49:39.404542
BoardName.BFS_LONGEST
5000 Evaluations: Average connections for bfs_long = 4.5692, std=0.6511615467762205
2023-05-01 19:57:41.279373
BoardName.RANDOM_WALK
5000 Evaluations: Average connections for random_walk = 4.5184, std=0.6911305520666845
2023-05-01 20:04:42.876065
BoardName.RANDOM_SEED
5000 Evaluations: Average connections for random_seed = 3.5536, std=1.0727194600640002
2023-05-01 20:13:50.694431
BoardName.NUMBERLINK
5000 Evaluations: Average connections for numberlink = 3.7418, std=0.9967611348763554
2023-05-01 20:27:37.145910
5000 Evaluations: Total average connections for parallel_randomwalk = 4.165066666666666, std=0.9359592203842121
CORRELATIONS
mean keys =  2146472881.4621334
mean episodes =  2499.5
Correlation of connections with detours is  -0.3158855014035944
Correlation of connections with keys is  0.0015189487555360432
Correlation of connections with episodes is  0.0036265839393111567
Correlation of connections with densities is  -0.371962895523934
Correlation of connections with diversities is  0.09368845123762706


In [ ]:
EVALUATING AN AGENT TRAINED ON RANDOM SEED

AGENT = ONLINE_RANDOMSEED_RANDOM=0.85_EXT21
(COMPARED TO AGENT TRAINED ON UNIFORM)

AGENT =  online_randomseed_100epochs_random085_ext21
2023-05-01 07:06:00.938634
BoardName.BFS_BASE
5000 Evaluations: Average connections for bfs_base = 3.8354, std=1.0211301777932136
2023-05-01 07:13:26.743334
BoardName.BFS_SHORTEST
5000 Evaluations: Average connections for bfs_short = 3.6768, std=1.0405487782896101
2023-05-01 07:21:34.631249
BoardName.BFS_LONGEST
5000 Evaluations: Average connections for bfs_long = 4.0956, std=0.9372623112021522
2023-05-01 07:29:27.201574
BoardName.RANDOM_WALK
5000 Evaluations: Average connections for random_walk = 4.0614, std=0.9353234948401542
2023-05-01 07:36:43.942145
BoardName.RANDOM_SEED
5000 Evaluations: Average connections for random_seed = 3.13, std=1.1707689780652715
2023-05-01 07:45:40.501084
BoardName.NUMBERLINK
5000 Evaluations: Average connections for numberlink = 3.0174, std=1.1353841816759647
2023-05-01 07:59:06.477342
5000 Evaluations: Total average connections for online_randomseed_100epochs_random085_ext21 = 3.6361, std=1.1262667490430498
CORRELATIONS
mean keys =  2143228144.9138
mean episodes =  2499.5
Correlation of connections with detours is  -0.3142740114616519
Correlation of connections with keys is  -0.008094080000827635
Correlation of connections with episodes is  -0.0065849812342847
Correlation of connections with densities is  -0.34311978928204045

AGENT =  uniform
2023-05-01 07:59:08.364544
BoardName.BFS_BASE
5000 Evaluations: Average connections for bfs_base = 4.5642, std=0.6094902460253159
2023-05-01 08:06:09.266864
BoardName.BFS_SHORTEST
5000 Evaluations: Average connections for bfs_short = 4.4352, std=0.6819097887550816
2023-05-01 08:14:07.866613
BoardName.BFS_LONGEST
5000 Evaluations: Average connections for bfs_long = 4.7206, std=0.5131623914512832
2023-05-01 08:21:22.295594
BoardName.RANDOM_WALK
5000 Evaluations: Average connections for random_walk = 4.6626, std=0.542181925187478
2023-05-01 08:28:04.478468
BoardName.RANDOM_SEED
5000 Evaluations: Average connections for random_seed = 3.8644, std=0.9631264922116929
2023-05-01 08:36:37.996060
BoardName.NUMBERLINK
5000 Evaluations: Average connections for numberlink = 4.0404, std=0.8901504591921525
2023-05-01 08:49:49.420510
5000 Evaluations: Total average connections for uniform = 4.381233333333333, std=0.78822235371048
CORRELATIONS
mean keys =  2145939502.3980668
mean episodes =  2499.5
Correlation of connections with detours is  -0.2985929884490893
Correlation of connections with keys is  -0.007405528493851191
Correlation of connections with episodes is  0.0009613393300615865
Correlation of connections with densities is  -0.3524013172227143






AGENT =  online_randomseed_random1_ext1
2023-04-28 07:39:10.450922
BoardName.BFS_BASE
5000 Evaluations: Average connections for bfs_base = 3.139, std=1.1506863169430668
2023-04-28 07:47:55.248753
BoardName.BFS_SHORTEST
5000 Evaluations: Average connections for bfs_short = 2.9808, std=1.1684311532991578
2023-04-28 07:56:34.286941
BoardName.BFS_LONGEST
5000 Evaluations: Average connections for bfs_long = 3.3928, std=1.1349485274672153
2023-04-28 08:05:09.439157
BoardName.RANDOM_WALK
5000 Evaluations: Average connections for random_walk = 3.3608, std=1.1335887084829313
2023-04-28 08:12:30.823453
BoardName.RANDOM_SEED
5000 Evaluations: Average connections for random_seed = 2.4876, std=1.2204287115599994
2023-04-28 08:21:09.540896
BoardName.NUMBERLINK
5000 Evaluations: Average connections for numberlink = 2.2632, std=1.1585878300759076
2023-04-28 08:34:38.998735
5000 Evaluations: Total average connections for online_randomseed_random1_ext1 = 2.9373666666666667, std=1.2369493652620636
CORRELATIONS
mean keys =  2152055817.0067
mean episodes =  2499.5
Correlation of connections with detours is  -0.3008370500743441
Correlation of connections with keys is  -0.0005197566954331764
Correlation of connections with episodes is  -0.0016811429412965646
Correlation of connections with densities is  -0.3157819883496236



In [ ]:
EFFECT OF INCREASING ITERATIONS (RANDOMNESS = 0.85)
AGENT =  uniform
2023-04-27 21:17:41.549702
ITERATIONS =  0
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 4.453, std=0.7069589804224853
2023-04-27 21:19:44.397174
ITERATIONS =  1
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 4.456, std=0.7099746474346813
2023-04-27 21:21:38.326788
ITERATIONS =  2
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 4.49, std=0.6956292115775472
2023-04-27 21:23:34.324253
ITERATIONS =  3
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 4.183, std=0.8303679907125515
2023-04-27 21:25:38.110841
ITERATIONS =  4
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 4.037, std=0.8795629596566696
2023-04-27 21:27:45.739394
ITERATIONS =  5
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 4.005, std=0.8665881374678517
2023-04-27 21:29:54.409347
ITERATIONS =  6
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.958, std=0.9177341663030749
2023-04-27 21:32:04.934304
ITERATIONS =  7
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.952, std=0.9141640990544313
2023-04-27 21:34:15.814106
ITERATIONS =  8
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.891, std=0.9214765325280944
2023-04-27 21:36:29.090983
ITERATIONS =  9
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.907, std=0.9393354033570755
2023-04-27 21:38:39.805515
ITERATIONS =  10
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.867, std=0.9226651613667876
2023-04-27 21:40:49.271900
1000 Evaluations: Total average connections for uniform = 4.109, std=0.8822651116715017
CORRELATIONS
mean keys =  2157879031.373182
mean episodes =  499.5
Correlation of connections with detours is  -0.1877113932851433
Correlation of connections with keys is  0.016329422128420912
Correlation of connections with episodes is  -0.01565463410235307
Correlation of connections with densities is  -0.16229764193494467

AGENT =  bfs_short
2023-04-27 21:40:50.096635
ITERATIONS =  0
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.824, std=1.2332980175123935
2023-04-27 21:43:17.483478
ITERATIONS =  1
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.869, std=1.173813869401789
2023-04-27 21:45:42.837600
ITERATIONS =  2
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.888, std=1.2318506402969476
2023-04-27 21:48:12.964781
ITERATIONS =  3
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.639, std=1.2118906716366786
2023-04-27 21:50:37.073113
ITERATIONS =  4
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.534, std=1.1877895436482002
2023-04-27 21:53:06.064112
ITERATIONS =  5
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.361, std=1.196109944779325
2023-04-27 21:55:33.154563
ITERATIONS =  6
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.415, std=1.167379544107228
2023-04-27 21:57:59.820916
ITERATIONS =  7
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.364, std=1.197290273910216
2023-04-27 22:00:29.260524
ITERATIONS =  8
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.373, std=1.173827500103827
2023-04-27 22:02:56.974167
ITERATIONS =  9
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.409, std=1.206531806460153
2023-04-27 22:05:24.251837
ITERATIONS =  10
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.344, std=1.1940117252355607
2023-04-27 22:07:53.973688
1000 Evaluations: Total average connections for bfs_short = 2.5472727272727274, std=1.2159327950110859
CORRELATIONS
mean keys =  2138519518.3032727
mean episodes =  499.5
Correlation of connections with detours is  -0.08954846244881376
Correlation of connections with keys is  0.007289860259366265
Correlation of connections with episodes is  -0.005472529105858147
Correlation of connections with densities is  -0.1250256245066308

AGENT =  random_walk
2023-04-27 22:07:54.767391
ITERATIONS =  0
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 4.237, std=0.9026798989675133
2023-04-27 22:09:51.321237
ITERATIONS =  1
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 4.254, std=0.906357545342896
2023-04-27 22:11:49.524749
ITERATIONS =  2
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 4.224, std=0.9131396388285858
2023-04-27 22:13:49.597965
ITERATIONS =  3
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.839, std=1.0387872736994808
2023-04-27 22:15:55.070838
ITERATIONS =  4
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.693, std=1.0792363040594957
2023-04-27 22:18:01.789320
ITERATIONS =  5
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.661, std=1.0982162810667124
2023-04-27 22:20:10.663296
ITERATIONS =  6
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.536, std=1.142236402851879
2023-04-27 22:22:20.815461
ITERATIONS =  7
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.527, std=1.128393105260751
2023-04-27 22:24:29.306231
ITERATIONS =  8
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.526, std=1.136364378181576
2023-04-27 22:26:41.269155
ITERATIONS =  9
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.533, std=1.1148591839331101
2023-04-27 22:28:49.460623
ITERATIONS =  10
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.558, std=1.109340344529126
2023-04-27 22:30:59.932061
1000 Evaluations: Total average connections for random_walk = 3.780727272727273, std=1.0961550231197883
CORRELATIONS
mean keys =  2140040993.6538181
mean episodes =  499.5
Correlation of connections with detours is  -0.20442743074294065
Correlation of connections with keys is  0.010876753808363228
Correlation of connections with episodes is  -0.0014988114886800808
Correlation of connections with densities is  -0.17510117878527756

AGENT =  numberlink
2023-04-27 22:31:00.716731
ITERATIONS =  0
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 4.168, std=0.8954194547808305
2023-04-27 22:32:56.622763
ITERATIONS =  1
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 4.256, std=0.8685988717469071
2023-04-27 22:34:56.119310
ITERATIONS =  2
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 4.168, std=0.8796453830948014
2023-04-27 22:36:55.432272
ITERATIONS =  3
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.871, std=1.0130937765083743
2023-04-27 22:39:03.995978
ITERATIONS =  4
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.758, std=1.0292890750416037
2023-04-27 22:41:11.416586
ITERATIONS =  5
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.601, std=1.0582055565909678
2023-04-27 22:43:23.809141
ITERATIONS =  6
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.516, std=1.106229632580867
2023-04-27 22:45:41.881835
ITERATIONS =  7
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.494, std=1.1081353707918542
2023-04-27 22:48:03.976303
ITERATIONS =  8
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.498, std=1.0779591829007256
2023-04-27 22:50:29.560225
ITERATIONS =  9
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.482, std=1.0647422223242582
2023-04-27 22:52:58.391615
ITERATIONS =  10
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.503, std=1.1144464993888221
2023-04-27 22:55:16.928149
1000 Evaluations: Total average connections for numberlink = 3.755909090909091, std=1.0653047327191028
CORRELATIONS

mean keys =  2152218527.8966365
mean episodes =  499.5
Correlation of connections with detours is  -0.19554870925376794
Correlation of connections with keys is  -0.0046873628040638125
Correlation of connections with episodes is  -0.012077733940820684
Correlation of connections with densities is  -0.17471901929220027

AGENT =  lsystems
2023-04-27 22:55:17.718382
ITERATIONS =  0
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.104, std=1.144195787441992
2023-04-27 22:57:48.245052
ITERATIONS =  1
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.125, std=1.1812599205932621
2023-04-27 23:00:27.172030
ITERATIONS =  2
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.141, std=1.1692386411678328
2023-04-27 23:03:01.914586
ITERATIONS =  3
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.705, std=1.2008226346967317
2023-04-27 23:05:34.566740
ITERATIONS =  4
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.484, std=1.1822622382534256
2023-04-27 23:08:08.924629
ITERATIONS =  5
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.568, std=1.1547190134400662
2023-04-27 23:10:50.868372
ITERATIONS =  6
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.503, std=1.1601685222414888
2023-04-27 23:13:31.479418
ITERATIONS =  7
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.406, std=1.2341653049733654
2023-04-27 23:16:15.505989
ITERATIONS =  8
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.356, std=1.1930062866556908
2023-04-27 23:18:54.247228
ITERATIONS =  9
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.382, std=1.1619277085946442
2023-04-27 23:21:33.918602
ITERATIONS =  10
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.459, std=1.120856369032179
2023-04-27 23:24:15.003406
1000 Evaluations: Total average connections for lsystems = 2.6575454545454544, std=1.2108965021185647
CORRELATIONS
mean keys =  2128751516.1352727
mean episodes =  499.5
Correlation of connections with detours is  -0.14243206879981835
Correlation of connections with keys is  0.0012770397734593825
Correlation of connections with episodes is  -0.006137274227336042
Correlation of connections with densities is  -0.18378896832983072




ITERATIONS=11 to 21

AGENT =  uniform
2023-04-27 23:47:07.715576
ITERATIONS =  11
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.815, std=0.9647668112036193
2023-04-27 23:49:22.928212
ITERATIONS =  12
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.858, std=0.912050437201803
2023-04-27 23:51:39.585024
ITERATIONS =  13
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.793, std=0.9413559369335279
2023-04-27 23:53:58.674226
ITERATIONS =  14
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.835, std=0.9642484119769137
2023-04-27 23:56:16.854753
ITERATIONS =  15
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.865, std=0.9245404263740985
2023-04-27 23:58:28.792182
ITERATIONS =  16
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.854, std=0.9448195594927108
2023-04-28 00:00:40.300833
ITERATIONS =  17
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.917, std=0.9187551360400658
2023-04-28 00:02:51.584119
ITERATIONS =  18
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.909, std=0.9190859589831628
2023-04-28 00:05:08.525346
ITERATIONS =  19
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.853, std=0.9630114225698468
2023-04-28 00:07:24.624361
ITERATIONS =  20
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.896, std=0.9094965640396889
2023-04-28 00:09:39.092329
ITERATIONS =  21
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.852, std=0.9508396289595843
2023-04-28 00:11:55.492693
1000 Evaluations: Total average connections for uniform = 3.8588181818181817, std=0.9384583410313106
CORRELATIONS
mean keys =  2163211841.6300907
mean episodes =  499.5
Correlation of connections with detours is  0.04605250234470573
Correlation of connections with keys is  -0.0013099834257932377
Correlation of connections with episodes is  -0.017181017339327226
Correlation of connections with densities is  0.07578132023170188

AGENT =  bfs_short
2023-04-28 00:11:56.306537
ITERATIONS =  11
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.307, std=1.1448803430926744
2023-04-28 00:14:31.917160
ITERATIONS =  12
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.354, std=1.1699076886660758
2023-04-28 00:17:01.228838
ITERATIONS =  13
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.304, std=1.1115682615116356
2023-04-28 00:19:40.121208
ITERATIONS =  14
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.398, std=1.1530810899498787
2023-04-28 00:22:16.178697
ITERATIONS =  15
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.355, std=1.166608331874927
2023-04-28 00:24:52.872764
ITERATIONS =  16
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.337, std=1.1787412778044213
2023-04-28 00:27:28.733647
ITERATIONS =  17
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.406, std=1.164973819448317
2023-04-28 00:30:02.594791
ITERATIONS =  18
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.429, std=1.1309106949710925
2023-04-28 00:32:36.315232
ITERATIONS =  19
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.411, std=1.1216412082301541
2023-04-28 00:35:07.696964
ITERATIONS =  20
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.318, std=1.1648502049619942
2023-04-28 00:37:40.339132
ITERATIONS =  21
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.395, std=1.1793960318739418
2023-04-28 00:40:14.619931
1000 Evaluations: Total average connections for bfs_short = 2.364909090909091, std=1.154329987052344
CORRELATIONS
mean keys =  2148499012.1273637
mean episodes =  499.5
Correlation of connections with detours is  0.10009770457111862
Correlation of connections with keys is  -0.011476674814272727
Correlation of connections with episodes is  -0.0018174932454193347
Correlation of connections with densities is  0.057722237631735715

AGENT =  random_walk
2023-04-28 00:40:15.398540
ITERATIONS =  11
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.519, std=1.1115930010574913
2023-04-28 00:42:27.442646
ITERATIONS =  12
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.516, std=1.0889187297498375
2023-04-28 00:44:41.977650
ITERATIONS =  13
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.486, std=1.1356953816935242
2023-04-28 00:46:53.572318
ITERATIONS =  14
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.444, std=1.1379209111357431
2023-04-28 00:49:04.640751
ITERATIONS =  15
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.487, std=1.1557815537548608
2023-04-28 00:51:16.827416
ITERATIONS =  16
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.533, std=1.1388200033367872
2023-04-28 00:53:32.426588
ITERATIONS =  17
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.505, std=1.1462874857556458
2023-04-28 00:55:47.071187
ITERATIONS =  18
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.441, std=1.1638380471526095
2023-04-28 00:58:06.852316
ITERATIONS =  19
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.522, std=1.1034110748039463
2023-04-28 01:00:24.829574
ITERATIONS =  20
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.542, std=1.116349407667689
2023-04-28 01:02:42.063401
ITERATIONS =  21
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.462, std=1.152630036047994
2023-04-28 01:04:59.845662
1000 Evaluations: Total average connections for random_walk = 3.496090909090909, std=1.1326490232712672
CORRELATIONS
mean keys =  2171570308.384
mean episodes =  499.5
Correlation of connections with detours is  0.024956766333404257
Correlation of connections with keys is  -0.008022235445578113
Correlation of connections with episodes is  -0.002897842009330285
Correlation of connections with densities is  0.03948192253558519

AGENT =  numberlink
2023-04-28 01:05:00.624572
ITERATIONS =  11
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.493, std=1.1180120750689593
2023-04-28 01:07:20.966901
ITERATIONS =  12
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.468, std=1.0867271966781729
2023-04-28 01:09:38.683774
ITERATIONS =  13
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.461, std=1.0910907386647548
2023-04-28 01:12:00.640485
ITERATIONS =  14
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.503, std=1.092698952136406
2023-04-28 01:14:20.480550
ITERATIONS =  15
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.468, std=1.0728354953113735
2023-04-28 01:16:40.951305
ITERATIONS =  16
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.495, std=1.0779494422281595
2023-04-28 01:19:02.906112
ITERATIONS =  17
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.505, std=1.104524784692494
2023-04-28 01:21:24.356810
ITERATIONS =  18
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.467, std=1.1273468853906503
2023-04-28 01:23:47.887013
ITERATIONS =  19
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.477, std=1.1024840134895382
2023-04-28 01:26:08.772480
ITERATIONS =  20
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.537, std=1.1084362859452048
2023-04-28 01:28:31.856789
ITERATIONS =  21
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.467, std=1.104043024523954
2023-04-28 01:30:54.647565
1000 Evaluations: Total average connections for numberlink = 3.4855454545454547, std=1.099078197535335
CORRELATIONS

mean keys =  2140601143.275
mean episodes =  499.5
Correlation of connections with detours is  0.029451729833068988
Correlation of connections with keys is  -0.003803276684529993
Correlation of connections with episodes is  -0.014976761295550808
Correlation of connections with densities is  0.0593016601869102

AGENT =  lsystems
2023-04-28 01:30:55.420840
ITERATIONS =  11
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.366, std=1.2033469989990417
2023-04-28 01:33:35.120768
ITERATIONS =  12
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.381, std=1.1789143310690562
2023-04-28 01:36:13.958532
ITERATIONS =  13
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.438, std=1.1958912994081026
2023-04-28 01:38:52.159956
ITERATIONS =  14
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.484, std=1.1890096719539331
2023-04-28 01:41:30.176945
ITERATIONS =  15
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.423, std=1.1610645976861065
2023-04-28 01:44:02.219952
ITERATIONS =  16
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.457, std=1.171388492345729
2023-04-28 01:46:31.798174
ITERATIONS =  17
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.445, std=1.1953974234538067
2023-04-28 01:49:07.017740
ITERATIONS =  18
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.47, std=1.1971215477135142
2023-04-28 01:51:44.477794
ITERATIONS =  19
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.516, std=1.1856407550350148
2023-04-28 01:54:17.082364
ITERATIONS =  20
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.501, std=1.1304861785975096
2023-04-28 01:56:52.033872
ITERATIONS =  21
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.437, std=1.1670608381742573
2023-04-28 01:59:26.653461
1000 Evaluations: Total average connections for lsystems = 2.447090909090909, std=1.1805702362183406
CORRELATIONS
mean keys =  2131853588.9318182
mean episodes =  499.5
Correlation of connections with detours is  0.08020111246212475
Correlation of connections with keys is  0.009108187658751026
Correlation of connections with episodes is  -0.0027782133855270134
Correlation of connections with densities is  0.018436515273952032






In [ ]:
EFFECT OF VARYING RANDOMNESS ON DIFFICULTY OF BOARDS
BUT ONLY 100 EVALUATIONS.  NOT ENOUGH.

AGENT =  uniform
2023-04-27 12:29:42.481720
RANDOMNESS =  0.0
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 4.07, std=0.851528038293514
2023-04-27 12:30:11.744234
RANDOMNESS =  0.2
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 3.92, std=0.9019977827023744
2023-04-27 12:30:40.660943
RANDOMNESS =  0.4
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 3.92, std=0.8681013765684281
2023-04-27 12:31:08.471889
RANDOMNESS =  0.6000000000000001
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 4.01, std=0.9433451118228154
2023-04-27 12:31:37.653821
RANDOMNESS =  0.8
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 3.84, std=0.9562426470305537
2023-04-27 12:32:06.387890
100 Evaluations: Total average connections for uniform = 3.952, std=0.9086781608468425
CORRELATIONS
mean keys =  2123742063.32
mean episodes =  49.5
Correlation of connections with detours is  0.0006230377283124016
Correlation of connections with keys is  -0.04040291085846616
Correlation of connections with episodes is  -0.03225318528800453
Correlation of connections with densities is  0.10042128970381678

AGENT =  bfs_short
2023-04-27 12:32:06.534606
RANDOMNESS =  0.0
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 2.4, std=1.3114877048604001
2023-04-27 12:32:38.462151
RANDOMNESS =  0.2
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 2.35, std=1.2114041439585717
2023-04-27 12:33:08.766875
RANDOMNESS =  0.4
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 2.21, std=1.168717245530329
2023-04-27 12:33:40.693133
RANDOMNESS =  0.6000000000000001
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 2.29, std=1.1161989070053777
2023-04-27 12:34:10.698548
RANDOMNESS =  0.8
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 2.44, std=1.2026637102698328
2023-04-27 12:34:41.648861
100 Evaluations: Total average connections for bfs_short = 2.338, std=1.2065471395681149
CORRELATIONS
mean keys =  2071304389.074
mean episodes =  49.5
Correlation of connections with detours is  0.040127154735123516
Correlation of connections with keys is  0.029243358684356787
Correlation of connections with episodes is  -0.027994496456962028
Correlation of connections with densities is  -0.06355071714367526

AGENT =  random_walk
2023-04-27 12:34:41.781718
RANDOMNESS =  0.0
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 3.74, std=1.0547037498748169
2023-04-27 12:35:10.987303
RANDOMNESS =  0.2
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 3.63, std=0.9965440281292143
2023-04-27 12:35:40.425833
RANDOMNESS =  0.4
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 3.66, std=1.1065260954898444
2023-04-27 12:36:08.759072
RANDOMNESS =  0.6000000000000001
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 3.6, std=1.1661903789690602
2023-04-27 12:36:38.004551
RANDOMNESS =  0.8
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 3.39, std=1.139254141971843
2023-04-27 12:37:07.445663
100 Evaluations: Total average connections for random_walk = 3.604, std=1.1005380502281599
CORRELATIONS
mean keys =  2069023301.978
mean episodes =  49.5
Correlation of connections with detours is  -0.0014610346541959686
Correlation of connections with keys is  0.08742786169773888
Correlation of connections with episodes is  -0.009380447137493777
Correlation of connections with densities is  0.10457479995457367

AGENT =  numberlink
2023-04-27 12:37:07.577889
RANDOMNESS =  0.0
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 3.7, std=0.9433981132056604
2023-04-27 12:37:37.328156
RANDOMNESS =  0.2
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 3.66, std=1.060377291344925
2023-04-27 12:38:07.288507
RANDOMNESS =  0.4
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 3.68, std=1.0571660229122009
2023-04-27 12:38:36.370159
RANDOMNESS =  0.6000000000000001
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 3.59, std=1.0871522432483869
2023-04-27 12:39:06.315704
RANDOMNESS =  0.8
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 3.42, std=1.159137610467368
2023-04-27 12:39:35.744557
100 Evaluations: Total average connections for numberlink = 3.61, std=1.068597211300872
CORRELATIONS
mean keys =  2156502997.742
mean episodes =  49.5
Correlation of connections with detours is  0.00822071052421733
Correlation of connections with keys is  -0.1007668153963554
Correlation of connections with episodes is  -0.054755525575147776
Correlation of connections with densities is  0.11485135491929124

AGENT =  lsystems
2023-04-27 12:39:35.880196
RANDOMNESS =  0.0
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 2.57, std=1.2186467905016614
2023-04-27 12:40:09.339157
RANDOMNESS =  0.2
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 2.53, std=1.2605950975630518
2023-04-27 12:40:40.465252
RANDOMNESS =  0.4
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 2.67, std=1.2004582458378135
2023-04-27 12:41:13.681859
RANDOMNESS =  0.6000000000000001
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 2.53, std=1.1353853971229328
2023-04-27 12:41:46.208440
RANDOMNESS =  0.8
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 2.54, std=1.2443472184241824
2023-04-27 12:42:19.149017
100 Evaluations: Total average connections for lsystems = 2.568, std=1.213827005796131
CORRELATIONS
mean keys =  2140709598.054
mean episodes =  49.5
Correlation of connections with detours is  0.11267256895514584
Correlation of connections with keys is  -0.022635759568808193
Correlation of connections with episodes is  0.03967074381413648
Correlation of connections with densities is  0.0359749595882062


NUMPY AGENTS ON ALL THE BOARDS WITH CORRELATIONS

AGENT =  uniform
2023-04-27 01:38:02.818085
BoardName.BFS_BASE
5000 Evaluations: Average connections for bfs_base = 4.5526, std=0.6213157329409904
2023-04-27 01:44:39.061435
BoardName.BFS_SHORTEST
5000 Evaluations: Average connections for bfs_short = 4.433, std=0.6704558150989519
2023-04-27 01:52:13.106151
BoardName.BFS_LONGEST
5000 Evaluations: Average connections for bfs_long = 4.715, std=0.5217039390305578
2023-04-27 01:59:09.432143
BoardName.RANDOM_WALK
5000 Evaluations: Average connections for random_walk = 4.6748, std=0.5428120853481433
2023-04-27 02:05:33.613297
BoardName.RANDOM_SEED

/usr/local/lib/python3.10/dist-packages/jax/_src/ops/scatter.py:89: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=int32. In future JAX releases this will result in an error.
  warnings.warn("scatter inputs have incompatible types: cannot safely cast "

5000 Evaluations: Average connections for random_seed = 3.8484, std=0.9515342558205668
2023-04-27 02:13:29.178871
BoardName.NUMBERLINK
5000 Evaluations: Average connections for numberlink = 4.0126, std=0.9031285844219525
2023-04-27 02:26:10.815254
5000 Evaluations: Total average connections for uniform = 4.372733333333334, std=0.7929290818786312
CORRELATIONS
mean keys =  2141478808.8718333
mean episodes =  2499.5
Correlation of connections with detours is  -0.3043430472800306
Correlation of connections with keys is  0.0021147426817292522
Correlation of connections with episodes is  0.005312942267783601
Correlation of connections with densities is  -0.3609169004267374

AGENT =  bfs_short
2023-04-27 02:26:12.624943
BoardName.BFS_BASE
5000 Evaluations: Average connections for bfs_base = 2.9382, std=1.1651526766909133
2023-04-27 02:34:10.730764
BoardName.BFS_SHORTEST
5000 Evaluations: Average connections for bfs_short = 2.8004, std=1.1554046217667646
2023-04-27 02:42:41.053300
BoardName.BFS_LONGEST
5000 Evaluations: Average connections for bfs_long = 3.1554, std=1.1532783011918677
2023-04-27 02:51:07.922740
BoardName.RANDOM_WALK
5000 Evaluations: Average connections for random_walk = 3.0934, std=1.1728070770591386
2023-04-27 02:58:52.480920
BoardName.RANDOM_SEED
5000 Evaluations: Average connections for random_seed = 2.3412, std=1.1874268651163322
2023-04-27 03:08:00.446338
BoardName.NUMBERLINK
5000 Evaluations: Average connections for numberlink = 3.8424, std=0.9438020131362297
2023-04-27 03:20:53.089756
5000 Evaluations: Total average connections for bfs_short = 3.0285, std=1.2188879152735905
CORRELATIONS
mean keys =  2158922482.734667
mean episodes =  2499.5
Correlation of connections with detours is  0.14772532102739205
Correlation of connections with keys is  0.00345210321304468
Correlation of connections with episodes is  0.0036621962803893688
Correlation of connections with densities is  0.07403909554748482

AGENT =  random_walk
2023-04-27 03:20:54.938323
BoardName.BFS_BASE
5000 Evaluations: Average connections for bfs_base = 4.2936, std=0.8360616245229774
2023-04-27 03:28:02.523034
BoardName.BFS_SHORTEST
5000 Evaluations: Average connections for bfs_short = 4.153, std=0.9024361473256709
2023-04-27 03:35:59.995396
BoardName.BFS_LONGEST
5000 Evaluations: Average connections for bfs_long = 4.5598, std=0.6797234437622406
2023-04-27 03:43:27.248951
BoardName.RANDOM_WALK
5000 Evaluations: Average connections for random_walk = 4.5262, std=0.6940558767131073
2023-04-27 03:50:18.612485
BoardName.RANDOM_SEED
5000 Evaluations: Average connections for random_seed = 3.488, std=1.1373020706918633
2023-04-27 03:59:04.850276
BoardName.NUMBERLINK
5000 Evaluations: Average connections for numberlink = 3.6562, std=1.1000007090906805
2023-04-27 04:12:25.304601
5000 Evaluations: Total average connections for random_walk = 4.1128, std=0.9970336804742355
CORRELATIONS
mean keys =  2150297489.4328
mean episodes =  2499.5
Correlation of connections with detours is  -0.30260609388110443
Correlation of connections with keys is  -0.003455071061657189
Correlation of connections with episodes is  -0.005974614967576746
Correlation of connections with densities is  -0.35947118869083433

AGENT =  numberlink
2023-04-27 04:12:27.154864
BoardName.BFS_BASE
5000 Evaluations: Average connections for bfs_base = 4.3752, std=0.7715082371562859
2023-04-27 04:19:50.570324
BoardName.BFS_SHORTEST
5000 Evaluations: Average connections for bfs_short = 4.2092, std=0.8517249321230418
2023-04-27 04:28:03.044013
BoardName.BFS_LONGEST
5000 Evaluations: Average connections for bfs_long = 4.5708, std=0.6586253563293779
2023-04-27 04:35:41.012380
BoardName.RANDOM_WALK
5000 Evaluations: Average connections for random_walk = 4.46, std=0.728285658241325
2023-04-27 04:42:49.485739
BoardName.RANDOM_SEED
5000 Evaluations: Average connections for random_seed = 3.4458, std=1.1111536167425278
2023-04-27 04:51:57.277979
BoardName.NUMBERLINK
5000 Evaluations: Average connections for numberlink = 3.6776, std=1.056247243783386
2023-04-27 05:05:17.503815
5000 Evaluations: Total average connections for numberlink = 4.1231, std=0.9727348336862758
CORRELATIONS
mean keys =  2144390277.6921
mean episodes =  2499.5
Correlation of connections with detours is  -0.3225426422820274
Correlation of connections with keys is  -0.0017784807648642718
Correlation of connections with episodes is  -0.00047481457596557227
Correlation of connections with densities is  -0.3783451306493704

AGENT =  lsystems
2023-04-27 05:05:19.325002
BoardName.BFS_BASE
5000 Evaluations: Average connections for bfs_base = 3.2328, std=1.096815463056571
2023-04-27 05:13:52.160406
BoardName.BFS_SHORTEST
5000 Evaluations: Average connections for bfs_short = 2.972, std=1.098187597817422
2023-04-27 05:23:05.029783
BoardName.BFS_LONGEST
5000 Evaluations: Average connections for bfs_long = 3.676, std=1.026948878961363
2023-04-27 05:31:45.091908
BoardName.RANDOM_WALK
5000 Evaluations: Average connections for random_walk = 3.5324, std=1.039879916144167
2023-04-27 05:40:03.446062
BoardName.RANDOM_SEED
5000 Evaluations: Average connections for random_seed = 2.4232, std=1.169658822050259
2023-04-27 05:49:29.028068
BoardName.NUMBERLINK
5000 Evaluations: Average connections for numberlink = 2.7394, std=1.1523400713331113
2023-04-27 06:03:45.221117
5000 Evaluations: Total average connections for lsystems = 3.0959666666666665, std=1.181929382643293
CORRELATIONS
mean keys =  2141372274.1216333
mean episodes =  2499.5
Correlation of connections with detours is  -0.224315057132908
Correlation of connections with keys is  0.0005751917077445454
Correlation of connections with episodes is  -0.004305912911889493
Correlation of connections with densities is  -0.2977234082438335



AGENT =  uniform
2023-04-27 00:51:14.738551
BoardName.BFS_BASE
2000 Evaluations: Average connections for bfs_base = 4.5565, std=0.6227421215880615
2023-04-27 00:54:05.485229
BoardName.BFS_SHORTEST
2000 Evaluations: Average connections for bfs_short = 4.4545, std=0.6737430890183588
2023-04-27 00:57:14.682526
BoardName.BFS_LONGEST
2000 Evaluations: Average connections for bfs_long = 4.704, std=0.521904205769603
2023-04-27 01:00:11.413419
BoardName.RANDOM_WALK
2000 Evaluations: Average connections for random_walk = 4.655, std=0.5347663040992766
2023-04-27 01:02:53.516845
BoardName.RANDOM_SEED
2000 Evaluations: Average connections for random_seed = 3.8195, std=0.9412331007779103
2023-04-27 01:06:12.327947
BoardName.NUMBERLINK
2000 Evaluations: Average connections for numberlink = 4.025, std=0.9051933495115836
2023-04-27 01:11:26.095806
2000 Evaluations: Total average connections for uniform = 4.369083333333333, std=0.7918506759835187
CORRELATIONS
mean keys =  2151179503.337
mean episodes =  999.5
Correlation of connections with detours is  -0.3039394602550464
Correlation of connections with keys is  -0.010371968415504144
Correlation of connections with episodes is  -0.013039951422649666
Correlation of connections with densities is  -0.3642894264295563

AGENT =  bfs_short
2023-04-27 01:11:28.166212
BoardName.BFS_BASE
2000 Evaluations: Average connections for bfs_base = 2.9585, std=1.1767658008286952
2023-04-27 01:14:47.390867
BoardName.BFS_SHORTEST
2000 Evaluations: Average connections for bfs_short = 2.826, std=1.173764882759746
2023-04-27 01:18:20.636179
BoardName.BFS_LONGEST
2000 Evaluations: Average connections for bfs_long = 3.1795, std=1.1829115562881276
2023-04-27 01:21:49.808128
BoardName.RANDOM_WALK
2000 Evaluations: Average connections for random_walk = 3.096, std=1.1746420731439855
2023-04-27 01:25:01.520556
BoardName.RANDOM_SEED
2000 Evaluations: Average connections for random_seed = 2.1625, std=1.134060734705157
2023-04-27 01:28:48.544430
BoardName.NUMBERLINK
2000 Evaluations: Average connections for numberlink = 3.828, std=0.981537569326819
2023-04-27 01:34:16.742333
2000 Evaluations: Total average connections for bfs_short = 3.0084166666666667, std=1.2417108465294522
CORRELATIONS
mean keys =  2150255903.085
mean episodes =  999.5
Correlation of connections with detours is  0.12070753195707394
Correlation of connections with keys is  -0.013698554104696786
Correlation of connections with episodes is  -0.01668016219120098
Correlation of connections with densities is  0.03733892084608622

AGENT =  random_walk
2023-04-27 01:34:17.574740
BoardName.BFS_BASE


In [ ]:
EVALUATED AGENTS TRAINED ON RANDOMSEED


AGENTS TRAINED WITH INFINITE-STEPS AND RANDOMNESS=1, VARYING ITERATIONS
AGENT =  iterations1
2023-04-26 13:31:02.824958
BoardName.BFS_BASE
2000 Evaluations: Average connections for bfs_base = 1.119, std=0.9401271190642252
2023-04-26 13:34:30.998542
BoardName.BFS_SHORTEST
2000 Evaluations: Average connections for bfs_short = 0.9915, std=0.9232701392333664
2023-04-26 13:37:58.994153
BoardName.BFS_LONGEST
2000 Evaluations: Average connections for bfs_long = 1.3275, std=1.0350090579313787
2023-04-26 13:41:25.754858
BoardName.RANDOM_WALK
2000 Evaluations: Average connections for random_walk = 1.3115, std=1.0456900831508351
2023-04-26 13:44:46.164472
BoardName.RANDOM_SEED
2000 Evaluations: Average connections for random_seed = 1.3715, std=1.0571129315262395
2023-04-26 13:48:20.192809
BoardName.NUMBERLINK
2000 Evaluations: Average connections for numberlink = 0.866, std=0.8532549443161755
2023-04-26 13:53:58.876942
2000 Evaluations: Total average connections for iterations1 = 1.1645, std=0.9965472475837093

AGENT =  iterations2
2023-04-26 13:53:59.113240
BoardName.BFS_BASE
2000 Evaluations: Average connections for bfs_base = 1.112, std=0.9340535316565106
2023-04-26 13:57:20.927835
BoardName.BFS_SHORTEST
2000 Evaluations: Average connections for bfs_short = 0.9415, std=0.8843515986303185
2023-04-26 14:00:50.294252
BoardName.BFS_LONGEST
2000 Evaluations: Average connections for bfs_long = 1.247, std=1.003987549723601
2023-04-26 14:04:23.400428
BoardName.RANDOM_WALK
2000 Evaluations: Average connections for random_walk = 1.244, std=1.0116639758338735
2023-04-26 14:07:47.002479
BoardName.RANDOM_SEED
2000 Evaluations: Average connections for random_seed = 1.481, std=1.0773295688878126
2023-04-26 14:11:26.783384
BoardName.NUMBERLINK
2000 Evaluations: Average connections for numberlink = 0.544, std=0.7007595878758991
2023-04-26 14:17:14.841876
2000 Evaluations: Total average connections for iterations2 = 1.0949166666666668, std=0.9882176681896667

AGENT =  iterations5
2023-04-26 14:17:15.077602
BoardName.BFS_BASE
2000 Evaluations: Average connections for bfs_base = 1.091, std=0.9548397771354104
2023-04-26 14:20:39.352295
BoardName.BFS_SHORTEST
2000 Evaluations: Average connections for bfs_short = 0.9255, std=0.880312302538139
2023-04-26 14:24:07.499759
BoardName.BFS_LONGEST
2000 Evaluations: Average connections for bfs_long = 1.2975, std=1.0044868092712815
2023-04-26 14:27:39.386712
BoardName.RANDOM_WALK
2000 Evaluations: Average connections for random_walk = 1.2345, std=1.001254088630853
2023-04-26 14:31:03.072436
BoardName.RANDOM_SEED
2000 Evaluations: Average connections for random_seed = 1.31, std=1.0741973747873341
2023-04-26 14:34:40.878467
BoardName.NUMBERLINK
2000 Evaluations: Average connections for numberlink = 0.777, std=0.8113390166878455
2023-04-26 14:40:20.602238
2000 Evaluations: Total average connections for iterations5 = 1.1059166666666667, std=0.9786206243426964

AGENT =  iterations6
2023-04-26 14:40:20.836595
BoardName.BFS_BASE
2000 Evaluations: Average connections for bfs_base = 1.1925, std=0.9856184606631513
2023-04-26 14:43:51.937758
BoardName.BFS_SHORTEST
2000 Evaluations: Average connections for bfs_short = 1.1245, std=0.9370164086076616
2023-04-26 14:47:30.359968
BoardName.BFS_LONGEST
2000 Evaluations: Average connections for bfs_long = 1.367, std=1.0184846586964382
2023-04-26 14:51:09.893074
BoardName.RANDOM_WALK
2000 Evaluations: Average connections for random_walk = 1.322, std=1.0179960707193323
2023-04-26 14:55:03.477294
BoardName.RANDOM_SEED
2000 Evaluations: Average connections for random_seed = 1.2955, std=1.0378727041405413
2023-04-26 14:59:31.432176
BoardName.NUMBERLINK
2000 Evaluations: Average connections for numberlink = 1.351, std=0.9347721647545995
2023-04-26 15:06:11.125980
2000 Evaluations: Total average connections for iterations6 = 1.2754166666666666, std=0.9933423342713003

AGENT =  iterations8
2023-04-26 15:06:11.380268
BoardName.BFS_BASE
2000 Evaluations: Average connections for bfs_base = 1.0355, std=0.9398083581241443
2023-04-26 15:10:04.032445
BoardName.BFS_SHORTEST
2000 Evaluations: Average connections for bfs_short = 0.9725, std=0.8987456536751652
2023-04-26 15:14:00.171128
BoardName.BFS_LONGEST
2000 Evaluations: Average connections for bfs_long = 1.215, std=0.9908455984662797
2023-04-26 15:17:53.129872
BoardName.RANDOM_WALK
2000 Evaluations: Average connections for random_walk = 1.266, std=1.0006218066782273
2023-04-26 15:21:51.414278
BoardName.RANDOM_SEED
2000 Evaluations: Average connections for random_seed = 0.997, std=0.9433933432031414
2023-04-26 15:26:15.827504
BoardName.NUMBERLINK
2000 Evaluations: Average connections for numberlink = 1.0905, std=0.9794435920460146
2023-04-26 15:32:43.474826
2000 Evaluations: Total average connections for iterations8 = 1.0960833333333333, std=0.9656697121974757

In [ ]:
COMPARING TIMES WITH AND WITHOUT BFS OPTIMISATION (1 ITERATION OF EXTENSION)

WITH EXTENSION =  7MIN 38SEC, 7min 21sec

AGENT =  uniform
2023-04-26 11:41:18.443837
BoardName.RANDOM_SEED
5000 Evaluations: Average connections for random_seed = 4.4606, std=0.692565982416116
2023-04-26 11:48:56.098630
        
AGENT =  uniform
2023-04-26 11:52:39.877301
BoardName.RANDOM_SEED
5000 Evaluations: Average connections for random_seed = 4.4606, std=0.692565982416116
2023-04-26 12:00:10.555290



WITHOUT EXTENSION = 6 MIN 47 SEC

AGENT =  uniform
2023-04-26 10:35:12.227726
BoardName.RANDOM_SEED
5000 Evaluations: Average connections for random_seed = 4.4606, std=0.692565982416116
2023-04-26 10:41:59.222164




TEST RESULTS OF NUMPY AGENTS ON (ALMOST) ALL BOARDS
AGENT =  uniform
2023-04-25 23:47:31.936717
BoardName.BFS_BASE
5000 Evaluations: Average connections for bfs_base = 4.5616, std=0.6156341770889592
2023-04-25 23:54:36.309006
BoardName.BFS_MIN_BENDS
5000 Evaluations: Average connections for bfs_min_bend = 4.5048, std=0.6548106291134865
2023-04-26 00:02:09.791877
BoardName.BFS_SHORTEST
5000 Evaluations: Average connections for bfs_short = 4.4262, std=0.6906182447633424
2023-04-26 00:09:17.828194
BoardName.BFS_LONGEST
5000 Evaluations: Average connections for bfs_long = 4.7214, std=0.5061442877283118
2023-04-26 00:15:49.293854
BoardName.RANDOM_WALK
5000 Evaluations: Average connections for random_walk = 4.6738, std=0.5427647372481009
2023-04-26 00:22:15.042061
BoardName.RANDOM_SEED
5000 Evaluations: Average connections for random_seed = 3.8534, std=0.9589100270619763
2023-04-26 00:30:11.145025
BoardName.WFC
5000 Evaluations: Average connections for wfc = 4.8294, std=0.4141203206798719
2023-04-26 01:02:28.658679
BoardName.NUMBERLINK
5000 Evaluations: Average connections for numberlink = 4.0318, std=0.904206149061153
2023-04-26 01:15:28.361175

AGENT =  bfs_short
2023-04-26 01:15:28.466341
BoardName.BFS_BASE
5000 Evaluations: Average connections for bfs_base = 2.939, std=1.1677666718998276
2023-04-26 01:23:25.850473
BoardName.BFS_MIN_BENDS
5000 Evaluations: Average connections for bfs_min_bend = 2.8648, std=1.1696670295430234
2023-04-26 01:31:55.247815
BoardName.BFS_SHORTEST
5000 Evaluations: Average connections for bfs_short = 2.8002, std=1.1627037283848367
2023-04-26 01:40:12.851809
BoardName.BFS_LONGEST
5000 Evaluations: Average connections for bfs_long = 3.1776, std=1.1627803919915403
2023-04-26 01:48:25.498068
BoardName.RANDOM_WALK
5000 Evaluations: Average connections for random_walk = 3.0686, std=1.162193632748003
2023-04-26 01:56:24.193549
BoardName.RANDOM_SEED
5000 Evaluations: Average connections for random_seed = 2.1476, std=1.1186662773141953
2023-04-26 02:05:27.206038
BoardName.WFC
5000 Evaluations: Average connections for wfc = 4.5914, std=0.6440854291163556
2023-04-26 02:38:20.015540
BoardName.NUMBERLINK
5000 Evaluations: Average connections for numberlink = 3.8352, std=0.965215499253923
2023-04-26 02:51:39.969372

AGENT =  random_walk
2023-04-26 02:51:40.081084
BoardName.BFS_BASE
5000 Evaluations: Average connections for bfs_base = 4.3218, std=0.8172176943752503
2023-04-26 02:59:02.256643
BoardName.BFS_MIN_BENDS
5000 Evaluations: Average connections for bfs_min_bend = 4.2388, std=0.8577730235907398
2023-04-26 03:06:56.172927
BoardName.BFS_SHORTEST
5000 Evaluations: Average connections for bfs_short = 4.1624, std=0.8768273718355285
2023-04-26 03:14:58.474362
BoardName.BFS_LONGEST
5000 Evaluations: Average connections for bfs_long = 4.5598, std=0.6803116638717874
2023-04-26 03:22:21.284912
BoardName.RANDOM_WALK
5000 Evaluations: Average connections for random_walk = 4.5128, std=0.709814172864983
2023-04-26 03:29:39.383039
BoardName.RANDOM_SEED
5000 Evaluations: Average connections for random_seed = 3.4714, std=1.1265797974400216
2023-04-26 03:39:01.862322
BoardName.WFC
5000 Evaluations: Average connections for wfc = 4.7724, std=0.5001982007164759
2023-04-26 04:11:57.579422
BoardName.NUMBERLINK
5000 Evaluations: Average connections for numberlink = 3.644, std=1.109983783665329
2023-04-26 04:26:05.278556

AGENT =  numberlink
2023-04-26 04:26:05.399620
BoardName.BFS_BASE
5000 Evaluations: Average connections for bfs_base = 4.3372, std=0.7991846845379359
2023-04-26 04:33:49.242904
BoardName.BFS_MIN_BENDS
5000 Evaluations: Average connections for bfs_min_bend = 4.2562, std=0.8235056526824817
2023-04-26 04:42:13.055582
BoardName.BFS_SHORTEST
5000 Evaluations: Average connections for bfs_short = 4.2214, std=0.849695263020808
2023-04-26 04:50:33.067931
BoardName.BFS_LONGEST
5000 Evaluations: Average connections for bfs_long = 4.573, std=0.6656357862975818
2023-04-26 04:58:10.367446
BoardName.RANDOM_WALK
5000 Evaluations: Average connections for random_walk = 4.483, std=0.727812475848003
2023-04-26 05:05:50.925140
BoardName.RANDOM_SEED
5000 Evaluations: Average connections for random_seed = 3.4868, std=1.0906079772310489
2023-04-26 05:15:11.024084
BoardName.WFC
5000 Evaluations: Average connections for wfc = 4.7162, std=0.5444791639723232
2023-04-26 05:47:51.963054
BoardName.NUMBERLINK
5000 Evaluations: Average connections for numberlink = 3.6354, std=1.064925743890155
2023-04-26 06:01:37.529204

AGENT =  lsystems
2023-04-26 06:01:37.636882
BoardName.BFS_BASE
5000 Evaluations: Average connections for bfs_base = 3.2256, std=1.0895433171746776
2023-04-26 06:10:11.088043
BoardName.BFS_MIN_BENDS
5000 Evaluations: Average connections for bfs_min_bend = 3.1334, std=1.116603976349717
2023-04-26 06:18:48.654105
BoardName.BFS_SHORTEST
5000 Evaluations: Average connections for bfs_short = 2.9472, std=1.1124801840931822
2023-04-26 06:27:24.561674
BoardName.BFS_LONGEST
5000 Evaluations: Average connections for bfs_long = 3.6326, std=1.0486263586235087
2023-04-26 06:35:36.462384
BoardName.RANDOM_WALK
5000 Evaluations: Average connections for random_walk = 3.5122, std=1.0462557813460338
2023-04-26 06:43:48.704323
BoardName.RANDOM_SEED
5000 Evaluations: Average connections for random_seed = 2.4956, std=1.1701199254777264
2023-04-26 06:53:02.639827
BoardName.WFC
5000 Evaluations: Average connections for wfc = 3.9088, std=0.9796338907979858
2023-04-26 07:27:07.950632
BoardName.NUMBERLINK
5000 Evaluations: Average connections for numberlink = 2.7446, std=1.1298543445949127
2023-04-26 07:41:45.950318


RANDOMSEED BOARD TIME ESTIMATES

W/O RUNNING AGENTS
10000 BOARDS (21 ITERATIONS, INFINITE WIRE LENGTH) IN ABOUT 1.5 MINUTES 
.01 SEC/BOARD TO GENERATE

W/ AGENTS
1 BOARD PRECOMPILED = 7 SEC
2023-04-25 14:07:12.295666
1 Evaluations: Average connections for random_seed = 4.0, std=0.0
2023-04-25 14:07:19.009992

10 BOARDS = 7 sec
2023-04-25 14:10:05.607338
10 Evaluations: Average connections for random_seed = 3.4, std=1.1135528725660044
2023-04-25 14:10:12.976135

100 BOARDS = 16 SEC
2023-04-25 14:11:07.214522
2023-04-25 14:11:23.295869

1000 BOARDS = 1 MIN 36 sec = 0.1 sec/board to solve
2023-04-25 14:12:27.030828
1000 Evaluations: Average connections for random_seed = 3.909, std=0.9288266792033916
2023-04-25 14:14:03.844435





1 BOARD, PRE-COMPILED

2023-04-25 13:14:16.051197
BoardName.RANDOM_SEED
SEEDED BOARD
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=1/0)>
Extension iteration  Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=2/0)>
Extension Step  Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=3/0)>
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=2/0)>
Optimization
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=2/0)>
1 Evaluations: Average connections for random_seed = 4.0, std=0.0
2023-04-25 13:14:22.602786


7 ITERATIONS, COMPILATION TIME STILL AT 16, CONNECTIONS BARELY DOWN TO 3.87

2023-04-25 08:03:44.757613
BoardName.RANDOM_SEED
SEEDED BOARD
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=1/0)>
Extension iteration  Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=2/0)>
Extension Step  Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=3/0)>
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=2/0)>
Optimization
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=2/0)>
10000 Evaluations: Average connections for random_seed = 3.8715, std=0.930799521916508
2023-04-25 08:19:40.966279



DECLARING ALL VARIABLES STATIC HAD NO EFFECT
5 ITERATIONS, COMPILATION TIME UP TO 16, CONNECTIONS DOWN TO 3.9

2023-04-25 07:37:21.125275
BoardName.RANDOM_SEED
SEEDED BOARD
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=1/0)>
Extension iteration  Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=2/0)>
Extension Step  Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=3/0)>
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=2/0)>
Optimization
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=2/0)>
10000 Evaluations: Average connections for random_seed = 3.9009, std=0.9294510153848885
2023-04-25 07:53:02.718436







5 ITERATIONS, COMPILATION TIME UP TO 16, CONNECTIONS DOWN TO 3.9
2023-04-25 00:40:19.011315
BoardName.RANDOM_SEED
SEEDED BOARD
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=1/0)>
Extension iteration  Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=2/0)>
Extension Step  Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=3/0)>
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=2/0)>
Optimization
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=2/0)>
10000 Evaluations: Average connections for random_seed = 3.9009, std=0.9294510153848885
2023-04-25 00:56:22.599802





AFTER SLIGHT REFINEMENT, COMPILATION TIME FOR 3 ITERATIONS IS ~14 MINUTES
2023-04-25 00:12:48.872868
BoardName.RANDOM_SEED
SEEDED BOARD
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=1/0)>
Extension iteration  Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=2/0)>
Extension Step  Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=3/0)>
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=2/0)>
Optimization
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=2/0)>

/usr/local/lib/python3.10/dist-packages/jax/_src/ops/scatter.py:89: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=int32. In future JAX releases this will result in an error.
  warnings.warn("scatter inputs have incompatible types: cannot safely cast "

10000 Evaluations: Average connections for random_seed = 4.0198, std=0.888823919570125
2023-04-25 00:26:39.313537







COMPILATION TIME WITH NUM_ITERATIONS =1 IS ABOUT THE SAME AS =2

2023-04-24 22:04:12.410881
BoardName.RANDOM_SEED
SEEDED BOARD
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=1/0)>
Extension iteration  Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=2/0)>
Extension Step  Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=3/0)>
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=2/0)>
Optimization
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=2/0)>
10000 Evaluations: Average connections for random_seed = 4.451, std=0.7166582170044519
2023-04-24 22:19:36.665689




COMPILE 
solved_board = solved_board_compiled(subkey, randomness = 1.0, two_sided = True,extension_iterations=int(2), max_extension=1e23)
2023-04-24 14:08:33.452352
BoardName.RANDOM_SEED
10000 Evaluations: Average connections for random_seed = 4.1619, std=0.8239468368772346
2023-04-24 14:23:46.241724






COMPILE WITH UGO'S FIX TO MAKE THE BFS OPTIMIZER JITABLE
2023-04-24 13:35:57.047568
BoardName.RANDOM_SEED
SEEDED BOARD
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=1/0)>
type(extension_iterations) = <class 'int'>
Extension iteration  1
Extension Step  Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=2/0)>
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=1/0)>

/usr/local/lib/python3.10/dist-packages/jax/_src/ops/scatter.py:89: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=int32. In future JAX releases this will result in an error.
  warnings.warn("scatter inputs have incompatible types: cannot safely cast "

Optimization
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=1/0)>
Extension iteration  2
Extension Step  Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=2/0)>
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=1/0)>
Optimization
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=1/0)>
10000 Evaluations: Average connections for random_seed = 4.1619, std=0.8239468368772346
2023-04-24 13:49:36.908408






COMPILE TIME AFTER REMOVING A COUPLE FOR LOOPS = 13MIN 24 SEC
2023-04-23 13:25:04.396975
BoardName.RANDOM_SEED
10000 Evaluations: Average connections for random_seed = 4.6052, std=0.6175216271516327
2023-04-23 13:38:28.168832




COMPILE TIME FOR FLATTENED INDEX (1D) = 27MIN 19 SEC
2023-04-23 08:58:40.162166
BoardName.RANDOM_SEED
num_extendables =  Traced<ShapedArray(int32[])>with<DynamicJaxprTrace(level=1/0)>

2023-04-23 09:12:51.532586: E external/xla/xla/service/slow_operation_alarm.cc:133] The operation took 10m11.74190574s

********************************
[Compiling module jit_return_solved_board] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************

10000 Evaluations: Average connections for random_seed = 4.6052, std=0.6175216271516327
2023-04-23 09:25:59.110785
        
        
        
COMPILE TIME FOR NESTED ROW/COL FORI_LOOPS = 24MIN 30 SEC
2023-04-23 09:39:07.191526
BoardName.RANDOM_SEED
num_extendables =  Traced<ShapedArray(int32[])>with<DynamicJaxprTrace(level=1/0)>

2023-04-23 09:44:25.666860: E external/xla/xla/service/slow_operation_alarm.cc:65] 
********************************
[Compiling module jit_return_solved_board] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************
2023-04-23 09:51:10.218737: E external/xla/xla/service/slow_operation_alarm.cc:133] The operation took 8m44.552012049s

********************************
[Compiling module jit_return_solved_board] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************

10000 Evaluations: Average connections for random_seed = 4.6052, std=0.6175216271516327
2023-04-23 10:03:37.019969


        
COMPILE TIME FOR NESTED FOR LOOPS = 25MIN 24 SEC
2023-04-23 11:42:12.463330
BoardName.RANDOM_SEED
num_extendables =  Traced<ShapedArray(int32[])>with<DynamicJaxprTrace(level=1/0)>

2023-04-23 11:47:36.265151: E external/xla/xla/service/slow_operation_alarm.cc:65] 
********************************
[Compiling module jit_return_solved_board] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************
2023-04-23 11:54:48.037782: E external/xla/xla/service/slow_operation_alarm.cc:133] The operation took 9m11.772777672s

********************************
[Compiling module jit_return_solved_board] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************

10000 Evaluations: Average connections for random_seed = 4.6052, std=0.6175216271516327
2023-04-23 12:07:36.375316


COMPARE RESULTS OF ONE-SIDED VS TWO-SIDED EXTENSIONS
TWO-SIDED = 13min 26sec
2023-04-21 14:22:49.151079
BoardName.RANDOM_SEED
10000 Evaluations: Average connections for random_seed = 4.4362, std=0.717446555500826
2023-04-21 14:36:15.766443

ONE-SIDED = 14min 43sec
2023-04-21 15:06:54.433743
BoardName.RANDOM_SEED
10000 Evaluations: Average connections for random_seed = 4.5804, std=0.6371309441551242
2023-04-21 15:21:37.132363

TWO-SIDED EXTENSION IS A LITTLE MORE DIFFICULT AND SLIGHTLY FASTER

TIME ESTIMATES FOR THEIR UNIFORM GENERATOR

10 BOARDS = 1.2 sec
2023-04-25 14:43:46.898485
10 Evaluations: Average connections for default Uniform board = 3.8, std=0.8717797887081347
2023-04-25 14:43:48.087526


100 BOARDS = 4 SEC
2023-04-25 14:42:45.907124
100 Evaluations: Average connections for default Uniform board = 4.14, std=0.8834025130143109
2023-04-25 14:42:49.580220

1000 BOARDS = 41.4 sec/30.5 SEC
2023-04-25 14:44:18.690546
1000 Evaluations: Average connections for default Uniform board = 4.048, std=0.8908961780140264
2023-04-25 14:44:50.098781

2023-04-25 20:28:41.685163
1000 Evaluations: Average connections for default Uniform board = 4.048, std=0.8908961780140264
2023-04-25 20:29:12.124470


# Save GIF

IT'S FASTER WITHOUT THE JAX.JIT() CALLS IN THE WIRE EXTENSION

JAX.JIT() CALL APPLIED TO BOTH THE EXTENSION AND PIN EXTRACTION
2023-04-18 23:17:35.299726
BoardName.RANDOM_WALK
1000 Evaluations: Average connections for random_walk = 4.514, std=0.6795616234014396
2023-04-18 23:22:00.377340

JAX.JIT() CALL APPLIED TO JUST THE PIN EXTRACTION
2023-04-18 23:22:53.809285
BoardName.RANDOM_WALK
1000 Evaluations: Average connections for random_walk = 4.486, std=0.6721636705446077
2023-04-18 23:24:32.408968

JAX.JIT() CALL APPLIED TO NEITHER
2023-04-18 23:25:10.222635
BoardName.RANDOM_WALK
1000 Evaluations: Average connections for random_walk = 4.484, std=0.655548625198772
2023-04-18 23:26:47.720919



In [ ]:
#env.animate(states, interval=150).save("./connector.gif")

# Save PNG

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

#env.render(states[-1])
state.grid = solved_board
env.render(state)
#plt.savefig("/home/randy/Downloads/randomseed_extended_jax00.png", dpi=300)
